In [22]:
import pandas as pd
import torch
import torch.nn.functional as F
import numpy as np

import csv
from torch.distributions import Categorical
np.set_printoptions(suppress=True)

In [2]:
import google.colab.drive
google.colab.drive.mount('/content/drive', force_remount=True)

root = '/content/drive/MyDrive/final'

Mounted at /content/drive


In [3]:
df = pd.read_csv(root + '/prob.csv')


In [4]:
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799
0,0,2.768213e-13,1.995840e-07,2.510389e-08,5.337574e-08,1.058253e-07,3.698659e-13,2.104603e-10,1.115754e-08,1.771728e-09,3.185358e-11,1.906086e-13,3.706901e-09,3.685393e-12,1.737325e-11,1.013478e-13,2.166627e-11,5.684173e-07,1.342649e-11,4.102874e-10,4.432971e-11,9.908679e-10,8.458840e-13,1.300746e-09,8.961718e-11,2.597715e-09,1.828161e-11,6.465203e-09,3.854678e-09,2.554534e-10,1.124631e-07,1.489750e-11,3.967093e-08,2.315819e-10,1.458455e-11,3.129647e-07,5.096085e-11,1.305021e-11,8.644358e-12,8.662456e-10,...,1.910805e-04,2.840356e-07,2.773619e-10,4.584139e-09,7.950167e-09,4.988038e-07,5.491048e-10,3.118798e-09,6.920217e-08,2.973011e-09,1.975594e-08,7.848991e-13,8.728469e-07,2.632204e-09,1.224356e-10,2.730909e-10,1.570893e-11,2.094352e-10,1.475581e-09,3.348698e-11,5.120926e-09,1.069764e-09,9.743920e-10,8.213209e-09,1.937599e-09,1.137860e-06,1.479682e-12,1.384833e-08,8.881649e-01,8.696222e-11,3.004397e-09,1.079085e-11,2.682221e-09,8.006940e-10,7.678790e-07,6.382189e-11,3.278833e-04,1.679800e-04,1.733014e-13,7.750870e-11
1,1,7.153312e-13,1.668120e-15,7.842528e-13,5.309132e-13,3.799460e-16,2.265874e-14,1.622544e-14,2.610626e-12,6.260894e-12,2.755936e-15,1.131512e-09,3.104628e-15,2.984317e-13,2.799543e-12,4.414705e-15,3.685138e-14,2.621089e-12,1.555315e-12,1.126177e-09,2.129994e-13,4.639755e-15,2.110438e-13,4.880815e-17,2.111513e-01,1.277204e-09,1.500389e-13,1.835801e-10,3.002697e-09,5.225438e-12,1.136449e-14,1.737639e-12,7.039883e-14,6.911396e-15,4.692739e-03,6.152420e-09,3.384092e-14,3.077690e-11,2.417106e-07,4.055574e-08,...,1.485438e-11,2.077510e-11,4.358446e-08,4.011758e-09,6.832973e-19,1.282136e-10,3.217730e-11,1.728362e-11,6.408071e-12,2.238804e-12,3.333970e-15,2.866446e-12,2.743297e-07,8.097738e-15,2.622251e-14,3.789872e-11,5.199808e-08,5.924236e-12,1.727914e-11,9.763643e-14,2.965940e-11,1.478187e-13,9.056395e-12,4.188252e-14,9.914853e-16,3.790805e-11,1.810989e-12,2.126038e-12,6.153917e-18,2.928957e-17,3.450892e-13,3.671027e-12,2.406011e-09,5.170405e-12,1.137749e-17,7.546275e-16,2.771454e-12,3.555452e-10,1.322831e-09,7.507300e-12
2,2,1.436208e-15,3.847444e-19,1.369024e-10,3.034063e-11,2.249675e-16,5.516310e-15,2.346528e-11,1.609041e-15,3.386531e-22,5.936821e-13,5.051861e-13,2.404141e-22,1.073757e-20,1.534197e-17,2.574922e-22,5.047946e-19,1.112077e-18,2.277257e-20,4.258672e-17,2.090907e-07,4.078399e-09,1.200279e-09,1.904052e-16,1.512759e-16,9.147095e-09,2.557524e-16,4.905438e-19,1.533036e-20,9.223816e-16,1.994936e-15,4.262973e-20,9.427844e-21,1.204833e-19,1.726444e-14,6.520324e-20,5.257036e-16,2.958145e-17,1.611092e-16,5.167341e-18,...,2.646220e-15,8.259876e-18,1.007530e-16,4.002757e-18,5.797160e-17,6.208078e-21,3.503250e-16,1.395428e-15,2.683776e-13,1.583787e-23,6.312985e-11,1.794785e-19,2.044612e-17,5.619281e-21,3.119710e-15,5.134462e-13,7.939190e-14,7.306946e-16,1.029672e-19,9.143240e-09,3.525026e-13,1.265477e-16,1.090423e-16,7.669942e-18,1.081635e-10,9.275665e-20,1.143251e-15,8.994659e-16,7.638168e-20,1.320402e-10,3.850521e-15,6.628084e-20,3.242311e-14,3.589416e-11,5.543240e-18,1.684384e-14,1.471698e-14,2.602107e-21,3.750988e-14,1.185937e-19
3,3,3.113767e-11,3.074713e-05,1.926557e-06,2.935371e-06,1.043562e-08,1.643748e-04,8.377590e-11,1.486564e-06,3.825746e-13,9.921943e-13,1.838673e-16,3.436160e-13,2.467498e-09,2.576067e-15,7.042679e-13,1.350618e-10,3.331013e-12,5.960492e-09,3.634938e-10,5.670924e-13,6.653460e-08,1.074440e-05,1.974462e-10,2.379616e-11,4.745033e-10,2.758078e-06,6.338481e-11,2.271939e-09,9.285451e-10,2.398584e-09,1.867780e-09,5.549152e-08,3.368168e-06,2.557201e-10,1.097827e-09,9.093064e-10,2.163284e-14,8.936755e-13,1.844680e-08,...,8.144740e-13,1.085483e-11,9.252441e-11,6.107885e-05,1.001648e-09,5.618417e-07,4.253337e-07,1.6861

In [3]:
# prob = torch.tensor(pd.read_csv(root + '/prob.csv').values)
# prob = prob[:,1:]

In [5]:
prob = torch.tensor(df.values)
prob = prob[:,1:]

In [4]:
n_request = 12800

In [36]:
# least confident
least_confident = torch.max(prob, dim=1).values
sorted_lc, index_lc = torch.sort(least_confident)
request_lc = index_lc[:n_request]

In [37]:
print(sorted_lc[:n_request])

tensor([0.0595, 0.0610, 0.0649,  ..., 0.1884, 0.1884, 0.1884],
       dtype=torch.float64)


In [38]:
np.savetxt(root + '/request_least_confident.csv', request_lc.numpy(), delimiter=',')

In [39]:
# margin sampling
sorted_prob, _ = torch.sort(prob, dim=1, descending=True)


In [40]:
ms = sorted_prob[:,0] - sorted_prob[:,1]
ms

tensor([0.8115, 0.3037, 0.9998,  ..., 0.7689, 0.1114, 0.0151],
       dtype=torch.float64)

In [41]:
sorted_ms, index_ms = torch.sort(ms)
request_ms = index_ms[:n_request]


In [12]:
print(sorted_ms[:n_request])

tensor([0.0000e+00, 5.6000e-07, 1.8800e-06,  ..., 1.1641e-02, 1.1643e-02,
        1.1643e-02], dtype=torch.float64)


In [42]:
np.savetxt(root + '/request_margin_sampling.csv', request_ms.numpy(), delimiter=',')

In [ ]:
# entropy v1
# entropy = torch.sum(-prob * torch.log(prob), dim=1)

In [ ]:
# sorted_entropy, index_entropy = torch.sort(entropy, descending=True)

In [7]:
prob.size()

torch.Size([512000, 800])

In [7]:
# entropy v2
entropy1 = Categorical(probs=prob[:256000,:]).entropy()
entropy2 = Categorical(probs=prob[256000:512000,:]).entropy()

In [15]:
entropy = torch.cat((entropy1, entropy2), dim=0)
sorted_entropy, index_entropy = torch.sort(entropy, descending=True)
request_entropy = index_entropy[:n_request]

In [21]:
print(sorted_entropy[0:n_request], request_entropy)
entropy.size()


tensor([4.4953, 4.3971, 4.3707,  ..., 3.0168, 3.0168, 3.0167],
       dtype=torch.float64) tensor([235405, 319481,  11494,  ...,  42202, 477902, 321943])


torch.Size([512000])

In [33]:
print(request_entropy.numpy())

[235405 319481  11494 ...  42202 477902 321943]


In [35]:
np.savetxt(root + '/request_entropy.csv', request_entropy.numpy(), delimiter=',')

In [34]:
# pd.DataFrame(request_entropy.numpy()).to_csv(root + '/request_entropy.csv')